In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

from src.stop_grid_mapper import StopGridMapper
from src.stop_explorer import StopExplorer
from src.grid_partitioning import Grid

### Main code

In [ ]:
# Create a GeoDataFrame for the stops dataset.
path_stops = './data_simulator/huge_dataset/dataset_simulator_trajectories.compressed.parquet.stops.parquet'
stop_explorer = StopExplorer(path_stops)

display(stop_explorer.get_df_stops())
stop_explorer.get_df_stops().info()

### Materialize a uniform grid, with side of a given length, over the bounding box enclosing the stop segments.

In [ ]:
grid = Grid(grid_cell_length_meters = 50)
grid.compute_grid_over_geodata(stop_explorer.get_df_stops())
display(grid.get_grid())

# mappa = grid.generate_grid_map()
# mappa

Associate the stop segments to the cells of the grid.
 
Also, compute some statistics concerning the grid's cells and the pairs '(uid, cell_id)' in the process.

In [ ]:
# Instantiate the StopGridMapper class, which has the effect of computing the join between the centroids of the 
# stop segments and grid cells. This effectively maps each stop segment to the grid cell it falls into.
stop_grid_mapper = StopGridMapper(grid, stop_explorer)
display(stop_grid_mapper.get_join())

# Compute some aggregate statistics for each pair (cell_id, user_id).
stats_uid_cell = stop_grid_mapper.compute_statistics_cells_users()
display(stats_uid_cell)

# Compute some aggregate statistics for each grid's cell.
augmented_grid = stop_grid_mapper.compute_statistics_cells()
display(augmented_grid)

### Plot some heatmaps of the grid augmented with statistics concerning its cells.

In [ ]:
dic_fields_tooltip = {"num_users" : "# users",
                      "num_stops" : "# stops",
                      "mean_duration_mins" : "mean stop duration (mins)", 
                      "median_duration_mins" : "median stop duration (mins)"}


# Plot heatmap of the grid, where each cell is colored according to the number of unique users
# that have at least a stop in that cell.
mappa = stop_grid_mapper.generate_augmented_grid_heatmap('num_users', 'Number of users per cell', dic_fields_tooltip)
mappa.save('map_users.html')


# Plot heatmap of the grid, where each cell is colored according to the number of stops that fall in that cell.
mappa = stop_grid_mapper.generate_augmented_grid_heatmap('num_stops', 'Number of stops per cell', dic_fields_tooltip)
mappa.save('map_stops.html')